In [1]:
from LLMstoDataBase import LLMsToDataBase, SQLAgent
from open_ai_utils import enviar_promt_chat_completions_mode, get_embedding, simular_respuesta_generativa
from calcular_tokens import num_tokens_from_string, num_tokens_from_messages

import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

import pandas as pd
import openai
import numpy as np
import os

In [2]:
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

# probando el pipeline paso a paso
## Carga de metadatos de la BBDD

Si el archivo.pickle existe en el directorio con los metadatos nos dice que ya existe, de lo contrario, los extrae de la bbdd. LA BASE DE DATOS DEBE ESTAR LEVANTADA 

In [3]:
pagila = {
    'bbdd_name':'pagila', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

rna_central = {
    'bbdd_name': 'pfmegrnargs', 
    'user'     : 'reader', 
    'password' : 'NWDMCE5xdipIjRrp', 
    'host'     : 'hh-pgsql-public.ebi.ac.uk', 
    'port'     : '5433'
}


llm_to_pagila = LLMsToDataBase(**pagila)
llm_to_pagila.store_metadata_df_to_pickle()
#llm_to_pagila.close_connection()


llm_to_rna = LLMsToDataBase(**rna_central)
llm_to_rna.store_metadata_df_to_pickle()
#llm_to_rna.close_connection()


El archivo ya existe en: ../data/nlp_to_sql/pagila_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pagila_full_metadata.pickle

Proceso de almacenamiento de metadatos para pagila completado.

El archivo ya existe en: ../data/nlp_to_sql/pfmegrnargs_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pfmegrnargs_full_metadata.pickle

Proceso de almacenamiento de metadatos para pfmegrnargs completado.



In [4]:
data_path = '../data/nlp_to_sql/'

simple_md_df = pd.read_pickle(data_path + 'pagila_simple_metadata.pickle')   # pfmegrnargs   # pagila
full_md_df = pd.read_pickle(data_path + 'pagila_full_metadata.pickle')

display(simple_md_df.head())
display(full_md_df.head())

,metadata_str,md_str_tokens,embedding
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,"[-0.010331695899367332, 0.012449457310140133, ..."
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,"[-0.02983446605503559, 0.018869545310735703, 0..."
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,"[-0.0329330638051033, -0.005584632977843285, 0..."
store,"{""store"": [""store_id"", ""manager_staff_id"", ""ad...",20,"[-0.029191363602876663, 0.008534164167940617, ..."
address,"{""address"": [""address_id"", ""address"", ""address...",32,"[-0.027424100786447525, 0.024846235290169716, ..."


,metadata_str,md_str_tokens,embedding
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,"[-0.0006032844539731741, 0.024174265563488007,..."
language,"table_name:language\n{""col_name"": ""language_id...",103,"[0.0009828200563788414, 0.027820346876978874, ..."
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,"[-0.0035037612542510033, 0.0059963702224195, 0..."
store,"table_name:store\n{""col_name"": ""store_id"", ""ty...",139,"[-0.006375625263899565, 0.021333562210202217, ..."
address,"table_name:address\n{""col_name"": ""address_id"",...",271,"[-0.0031455650459975004, 0.02590346895158291, ..."


In [5]:
print('El numero de tablas que tiene la actual base de datos es: ')
print(len(simple_md_df), len(full_md_df))

El numero de tablas que tiene la actual base de datos es: 
15 15


## Buscando metadatos más similares a una consulta

In [6]:
# consulta facil - medio
consulta_pagila = '''
dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula
'''
# consulta facil
consulta_pagila_01 = '''cuantas peliculas tienen la duración maxima y cuanta es esa duración'''

# consulta dificil media
consulta_pagila_02 = '''Los nombres de los actores que hayan participado en más de 3 peliculas, \
                        cuyo nombre empieze por una vocal, y que las peliculas duren más de 180 minutos'''

#consulta dificil
consulta_pagila_03 = '''¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
                        devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?'''

# consulta_rna_central_1 = "Muestra todas las secuencias de ARN que tienen una longitud mayor a 200 nucleótidos."
# consulta_rna_central_2 =  "Encuentra las entradas de ARN que han sido identificadas en organismos específicos como 'Homo sapiens'."
# consulta_rna_central_3 = "Lista los estudios que han citado secuencias de ARN particulares más de 50 veces."
# consulta_rna_central_4 =  "Obtén los nombres de los investigadores que han contribuido con más de 10 secuencias de ARN a la base de datos."
# consulta_rna_central_5 = "Muestra las secuencias de ARN que se han descubierto en el último año."

token_limit = 1000
n_tablas = None

most_similar_md_simple = LLMsToDataBase.search_most_similar_metadata(
    prompt= consulta_pagila, 
    metadata=simple_md_df, 
    n_resultados=n_tablas, 
    lim_tokens=token_limit)

most_similar_md_full = LLMsToDataBase.search_most_similar_metadata(
    prompt= consulta_pagila, 
    metadata=full_md_df, 
    n_resultados=n_tablas, 
    lim_tokens=token_limit )

display(most_similar_md_simple)
display(most_similar_md_full)

,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
film_actor,"{""film_actor"": [""actor_id"", ""film_id"", ""last_u...",16,0.732440,"[-0.03354605287313461, 0.0032554285135120153, ...",16
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,0.732104,"[-0.010331695899367332, 0.012449457310140133, ...",69
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,0.725583,"[-0.0329330638051033, -0.005584632977843285, 0...",88
film_category,"{""film_category"": [""film_id"", ""category_id"", ""...",16,0.699950,"[-0.025598706677556038, 0.014962821267545223, ...",104
inventory,"{""inventory"": [""inventory_id"", ""film_id"", ""sto...",19,0.691989,"[-0.017722617834806442, 0.002165640937164426, ...",123
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,0.686570,"[-0.02983446605503559, 0.018869545310735703, 0...",137
rental,"{""rental"": [""rental_id"", ""rental_date"", ""inven...",34,0.683688,"[-0.02543514594435692, 0.002907260088250041, 0...",171
staff,"{""staff"": [""staff_id"", ""first_name"", ""last_nam...",42,0.676197,"[-0.01896057464182377, 0.008822894655168056, -...",213
category,"{""category"": [""category_id"", ""name"", ""last_upd...",14,0.674468,"[-0.030227964743971825, 0.017394820228219032, ...",227
address,"{""address"": [""address_id"", ""address"", ""address...",32,0.674146,"[-0.027424100786447525, 0.024846235290169716, ...",259


,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,0.742271,"[-0.0035037612542510033, 0.0059963702224195, 0...",138
film_actor,"table_name:film_actor\n{""col_name"": ""actor_id""...",107,0.740522,"[-0.00541864987462759, 0.010073642246425152, 0...",245
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,0.723561,"[-0.0006032844539731741, 0.024174265563488007,...",688
film_category,"table_name:film_category\n{""col_name"": ""film_i...",107,0.718146,"[0.0017659964505583048, 0.02148573473095894, -...",795
inventory,"table_name:inventory\n{""col_name"": ""inventory_...",138,0.709250,"[0.00014501469559036195, 0.015438942238688469,...",933


In [7]:
print(len(most_similar_md_simple))
print(len(most_similar_md_full))

15
5


In [8]:
simple_md_str = LLMsToDataBase.get_string_from_metadata_df(most_similar_md_simple)
full_md_str = LLMsToDataBase.get_string_from_metadata_df(most_similar_md_full)

print('SIMPLE METADATA')
print(simple_md_str)
print('\n')
print('FULL METADATA')
print(full_md_str)

SIMPLE METADATA
{"film_actor": ["actor_id", "film_id", "last_update"]}
{"film": ["film_id", "title", "description", "release_year", "language_id", "rental_duration", "rental_rate", "length", "replacement_cost", "rating", "last_update", "special_features", "fulltext"]}
{"actor": ["actor_id", "first_name", "last_name", "last_update"]}
{"film_category": ["film_id", "category_id", "last_update"]}
{"inventory": ["inventory_id", "film_id", "store_id", "last_update"]}
{"language": ["language_id", "name", "last_update"]}
{"rental": ["rental_id", "rental_date", "inventory_id", "customer_id", "return_date", "staff_id", "last_update"]}
{"staff": ["staff_id", "first_name", "last_name", "address_id", "email", "store_id", "active", "username", "password", "last_update", "picture"]}
{"category": ["category_id", "name", "last_update"]}
{"address": ["address_id", "address", "address2", "district", "city_id", "postal_code", "phone", "last_update"]}
{"city": ["city_id", "city", "country_id", "last_update

# Probando el agente

## Lo probamos en dos consultas

In [4]:
consulta ='''
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?
'''

agent = SQLAgent(**pagila)
# print('conexion')
# print(agent.bbdd_name)
# print(agent.historico.contador_interacciones)

tabla_resultado, codigo_sql = agent.nlp_to_sql(consulta_nlp=consulta, metadata_token_limit= 1000)


Voy a asegurar que los metadatos de bbdd pagila esten disponibles...

El archivo ya existe en: ../data/nlp_to_sql/pagila_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pagila_full_metadata.pickle

Proceso de almacenamiento de metadatos para pagila completado.


AGENTE:
El codigo sql que se ejecutará para responder tu consulta:

SELECT a.first_name, a.last_name, f.title, f.rental_rate
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE a.actor_id IN (
    SELECT actor_id
    FROM film_actor
    GROUP BY actor_id
    ORDER BY COUNT(film_id) DESC
    LIMIT 1
);



## Probando el metodo 'continuar_conversacion'

### Con una nueva consulta

In [5]:
continuar = '''dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula'''

cont, nueva_consulta = agent.continuar_conversando(
    usuario=continuar, 
    codigo_sql_ejecutado=codigo_sql, 
    tabla_consulta_anterior=tabla_resultado
)

print(cont, nueva_consulta)

True True


### Con una duda sobre la consulta anterior

In [6]:
duda = '''Cuales de los actores mencionados, tienen un nombre que empieza por una vocal'''

cont, nueva_consulta = agent.continuar_conversando(
    usuario= duda, 
    codigo_sql_ejecutado= '', 
    tabla_consulta_anterior= tabla_resultado
)

print(cont, nueva_consulta)

True False


### Texto random

In [7]:
duda = '''como se hacen los macarrones?'''

cont, nueva_consulta = agent.continuar_conversando(
    usuario=duda, 
    codigo_sql_ejecutado=codigo_sql, 
    tabla_consulta_anterior=tabla_resultado
)

print(cont, nueva_consulta)

False False


## añadimos una nueva consulta

In [8]:
consulta ='''dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula'''

#agent = SQLAgent(**pagila)
# print('conexion')
# print(agent.bbdd_name)
# print(agent.historico.contador_interacciones)

tabla_resultado, codigo_sql = agent.nlp_to_sql(consulta_nlp=consulta, metadata_token_limit= 1000)
agent.close_connection()
# print('connexion cerrada')

# # simular_respuesta_generativa(tabla_resultado)


AGENTE:
El codigo sql que se ejecutará para responder tu consulta:

SELECT a.first_name, a.last_name, f.length
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE f.length > 180;



## generar informe

In [9]:
informe = agent.informe_resultado(
    consulta_usuario=consulta, 
    codigo_sql= codigo_sql, 
    tabla_texto= tabla_resultado, 
    max_tokens_respuesta= 1000
)

simular_respuesta_generativa(informe)

El informe detalla los actores y actrices que han participado en películas con una duración superior a 180 minutos, incluyendo la duración exacta de cada película:

- Penelope Guiness, 184 minutos
- Ed Chase, 183 minutos
- Jennifer Davis, 182 minutos
- Johnny Lollobrigida, 183, 181, 185, 181 minutos
- Joe Swank, 181 (dos películas)
- Zero Cage, 184 minutos
- Uma Wood, 184, 185 minutos
- Cuba Olivier, 185, 184 minutos
- Fred Costner, 184 minutos
- Bob Fawcett, 185 (dos películas), 181 (dos películas)
- Lucille Tracy, 184 minutos
- Kirsten Paltrow, 185 minutos
- Elvis Marx, 185 minutos
- Sandra Kilmer, 185, 181, 184 minutos
- Rip Crawford, 183, 184 minutos
- Julia Mcqueen, 183 minutos
- Woody Hoffman, 184, 183 minutos
- Milla Peck, 181 minutos
- Audrey Olivier, 185 minutos
- Judy Dean, 185 minutos
- Burt Dukakis, 185, 184 minutos
- Val Bolger, 181 minutos
- Tom Mckellen, 181 minutos
- Johnny Cage, 183 minutos
- Jodie Degeneres, 185, 182 minutos
- Tom Miranda, 181 (dos películas)
- Kirk J

In [10]:
print(num_tokens_from_string(informe))

500


## Probando la duda sobre consutla anterior

In [11]:
duda = '''de los actores mencionados, dame los nombres de cuyos apellidos empiezen por una vocal'''

respuesta_duda = agent.pregunta_sobre_consulta_anterior(
    usuario= duda, 
    tabla_consulta_anterior=None, # le podemos pasar la tabla o no, lo más segura es pasarla pero tambien es más costoso 
    max_tokens_historico= 1500, 
    max_tokens_respuesta=300
)


print(respuesta_duda)

Los actores y actrices mencionados cuyos apellidos comienzan con una vocal son:

- Ed Chase
- Uma Wood
- Elvis Marx
- Audrey Olivier
- Uma Thurman
- Oprah Winfrey


## Probando el historico

### historico completo

In [12]:
print(agent.historico_completo.historico)
num_tokens_from_string(agent.historico_completo.historico)

prompt_1 USUARIO:
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas,             devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?

codigo SQL_1 AGENTE:
SELECT a.first_name, a.last_name, f.title, f.rental_rate
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE a.actor_id IN (
    SELECT actor_id
    FROM film_actor
    GROUP BY actor_id
    ORDER BY COUNT(film_id) DESC
    LIMIT 1
);

tabla_1 AGENTE:
|    | first_name   | last_name   | title                   |   rental_rate |
|---:|:-------------|:------------|:------------------------|--------------:|
|  0 | Gina         | Degeneres   | Bed Highball            |          2.99 |
|  1 | Gina         | Degeneres   | Calendar Gunfight       |          4.99 |
|  2 | Gina         | Degeneres   | Chamber Italian         |          4.99 |
|  3 | Gina         | Degeneres   | Chaplin License         |          2

5020

### Historico simple

Esto solo guarda las consultas del usuario y el informe generado por el LLM. Hay que tener en cuenta que en el flujo de dialogo real despues de una nueva consulta siempre se generará un informe por lo que siempre ira una respuesta después de una consulta del usuario

In [13]:
print(agent.historico.historico)

prompt_1 USUARIO:
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas,             devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas?

prompt_2 USUARIO:
dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula

respuesta_informe_2 AGENTE:
El informe detalla los actores y actrices que han participado en películas con una duración superior a 180 minutos, incluyendo la duración exacta de cada película:

- Penelope Guiness, 184 minutos
- Ed Chase, 183 minutos
- Jennifer Davis, 182 minutos
- Johnny Lollobrigida, 183, 181, 185, 181 minutos
- Joe Swank, 181 (dos películas)
- Zero Cage, 184 minutos
- Uma Wood, 184, 185 minutos
- Cuba Olivier, 185, 184 minutos
- Fred Costner, 184 minutos
- Bob Fawcett, 185 (dos películas), 181 (dos películas)
- Lucille Tracy, 184 minutos
- Kirsten Paltrow, 185 minutos
- Elvi

## Historico estructurado

Hay que tener en cuenta que se guarda la información estrucutrada cada vez que responde el LLM en este caso no ha sido así, por eso vemos la incoherencia en el indice o las llaves

In [15]:
for key, value in agent.historico_completo.info_consultas_estructurada.items():
    print(key)
    for n in value: 
        print(n)

2
dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula
|     | first_name   | last_name    |   length |
|----:|:-------------|:-------------|---------:|
|   0 | Penelope     | Guiness      |      184 |
|   1 | Ed           | Chase        |      183 |
|   2 | Jennifer     | Davis        |      182 |
|   3 | Johnny       | Lollobrigida |      183 |
|   4 | Johnny       | Lollobrigida |      181 |
|   5 | Johnny       | Lollobrigida |      185 |
|   6 | Johnny       | Lollobrigida |      181 |
|   7 | Joe          | Swank        |      181 |
|   8 | Joe          | Swank        |      181 |
|   9 | Zero         | Cage         |      184 |
|  10 | Uma          | Wood         |      184 |
|  11 | Uma          | Wood         |      185 |
|  12 | Cuba         | Olivier      |      185 |
|  13 | Cuba         | Olivier      |      184 |
|  14 | Fred         | Costner      |      18

In [9]:
consulta ='''¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas, \
            devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas cuyos nombres empiezen por las primeras tres letras del abecedario?'''

agent2 = SQLAgent(**pagila)
# print('conexion')
# print(agent.bbdd_name)
# print(agent.historico.contador_interacciones)

tabla_resultado, codigo_sql = agent2.nlp_to_sql(consulta_nlp=consulta, metadata_token_limit= 1000)
agent.close_connection()


Voy a asegurar que los metadatos de bbdd pagila esten disponibles...

El archivo ya existe en: ../data/nlp_to_sql/pagila_simple_metadata.pickle
El archivo ya existe en: ../data/nlp_to_sql/pagila_full_metadata.pickle

Proceso de almacenamiento de metadatos para pagila completado.


AGENTE:
El codigo sql que se ejecutará para responder tu consulta:
WITH ActorFilmCount AS (
    SELECT
        fa.actor_id,
        COUNT(fa.film_id) AS film_count
    FROM
        film_actor fa
    GROUP BY
        fa.actor_id
    ORDER BY
        film_count DESC
    LIMIT 1
),
ActorsWithMostFilms AS (
    SELECT
        a.actor_id,
        a.first_name,
        a.last_name
    FROM
        actor a
    WHERE
        a.actor_id IN (SELECT actor_id FROM ActorFilmCount)
        AND (a.first_name ~ '^A|^B|^C' OR a.last_name ~ '^A|^B|^C')
),
FilmPrices AS (
    SELECT
        f.film_id,
        f.title,
        f.rental_rate
    FROM
        film f
    JOIN
        film_actor fa ON f.film_id = fa.film_id
    WHE

In [11]:
print(agent2.historico_completo.historico)

prompt_1 USUARIO:
¿Que precio tienen las peliculas en las que ha trabajado el actor o actores con más películas,             devuelve el nombre de las películas y el actor o actores que hayan trabajado en ellas cuyos nombres empiezen por las primeras tres letras del abecedario?

codigo SQL_1 AGENTE:
WITH ActorFilmCount AS (
    SELECT
        fa.actor_id,
        COUNT(fa.film_id) AS film_count
    FROM
        film_actor fa
    GROUP BY
        fa.actor_id
    ORDER BY
        film_count DESC
    LIMIT 1
),
ActorsWithMostFilms AS (
    SELECT
        a.actor_id,
        a.first_name,
        a.last_name
    FROM
        actor a
    WHERE
        a.actor_id IN (SELECT actor_id FROM ActorFilmCount)
        AND (a.first_name ~ '^A|^B|^C' OR a.last_name ~ '^A|^B|^C')
),
FilmPrices AS (
    SELECT
        f.film_id,
        f.title,
        f.rental_rate
    FROM
        film f
    JOIN
        film_actor fa ON f.film_id = fa.film_id
    WHERE
        fa.actor_id IN (SELECT actor_id FROM A

## Le volvemos a pasar la tabla en formato texto al llm para que nos comente los resultado

In [ ]:
informe = LLMsToDataBase.informe_resultado(consulta_usuario= consulta_pagila_03, tabla_texto=tabla_resultado, max_tokens_respuesta=2000 )
simular_respuesta_generativa(informe)

TypeError: informe_resultado() missing 1 required positional argument: 'codigo_sql'

In [ ]:
llm_to_pagila.close_connection()

In [ ]:
from calcular_tokens import num_tokens_from_string

print(num_tokens_from_string(informe))
print(num_tokens_from_string(tabla_resultado))

378
1010
